# LTE Data Science Capstone Project
Develop a machine learning model to analyze and predict car accident severity


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
# Get Data
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

df = pd.read_csv('Data-Collisions.csv')

--2020-09-09 12:53:35--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  45.0MB/s   in 1.6s   

2020-09-09 12:53:37 (45.0 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



In [5]:
from sklearn.utils import resample

# Separate majority and minority classes
df_maj = df[df.SEVERITYCODE == 1]
df_min = df[df.SEVERITYCODE == 2]

# Upsample minority class
df_min_upsampled = resample(df_min, 
                            replace = True,     # sample with replacement
                            n_samples = 136485)    # to match majority class

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_maj, df_min_upsampled])
 
# Display new class counts
df_upsampled.SEVERITYCODE.value_counts()

2    136485
1    136485
Name: SEVERITYCODE, dtype: int64

# First Category Transformation: Hot Coding (binary)

In [ ]:
y = df_upsampled['SEVERITYCODE'].values
feature = df_upsampled[['WEATHER', 'ROADCOND', 'LIGHTCOND']]

feature = pd.get_dummies(feature, columns=['WEATHER', 'ROADCOND', 'LIGHTCOND']).head()

X = feature
X[0:5]

# Alternate Category Transformation: Find/Replace

In [6]:
feature = df_upsampled[['WEATHER', 'ROADCOND', 'LIGHTCOND']]
y = df_upsampled['SEVERITYCODE'].values


cleanup_nums = {'WEATHER':    {"Clear": 0, 'Raining': 1, 'Overcast': 2, 'Unknown': 3,
                              'Snowing': 4, 'Other': 5, 'Fog/Smog/Smoke': 6, 'Sleet/Hail/Freezing Rain': 7,
                              'Blowing Sand/Dirt': 8, 'Severe Crosswind': 9, 'Partly Cloudy': 10},
                'ROADCOND':   {"Dry": 0, "Wet": 1, "Unknown": 2, "Ice": 3,
                              "Snow/Slush": 4, "Other": 5, "Standing Water": 6,
                              'Sand/Mud/Dirt': 7, 'Oil': 8},
               'LIGHTCOND': {'Daylight': 0, 'Dark - Street Lights On': 1, 'Unknown': 2, 'Dusk': 3, 
                            'Dawn': 4, 'Dark - No Street Lights': 5, 'Dark - Street Lights Off': 6,
                            'Other': 7, 'Dark - Unknown Lighting':8}}

feature.replace(cleanup_nums, inplace=True)

X = feature

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [7]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 0.26195087,  0.79309055,  0.39297418],
       [ 1.20296861, -0.5896768 , -0.57005156],
       [-0.67906687, -0.5896768 , -0.57005156],
       [-0.67906687, -0.5896768 , -0.57005156],
       [ 0.26195087,  0.79309055, -0.57005156]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (191079, 3) (191079,)
Test set: (81891, 3) (81891,)


# K Nearest Neighbor

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

n = 10
#Ks = 10
#mean_acc = np.zeros((Ks-1))
#for n in range(1,Ks):
    
    #Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train, y_train)
#    yhat = neigh.predict(X_test)
#    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

#print("The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

#neigh = KNeighborsClassifier(n_neighbors = 7).fit(X_train,y_train)
#neigh

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.tree import DecisionTreeClassifier

debtTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
debtTree.fit(X_train,y_train)
yhatTree = debtTree.predict(X_test)

print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, yhatTree))

In [ ]:
df.SEVERITYCODE.value_counts()
#df.ROADCOND.value_counts()
#df.LIGHTCOND.value_counts()

In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
df_maj = df[df.SEVERITYCODE == 1]
df_min = df[df.SEVERITYCODE == 2]

# Upsample minority class
df_maj_dnsampled = resample(df_maj, 
                            replace = False,     # sample with replacement
                            n_samples = 58188)    # to match majority class

# Combine majority class with upsampled minority class
df_dnsampled = pd.concat([df_min, df_maj_dnsampled])
 
# Display new class counts
df_dnsampled.SEVERITYCODE.value_counts()

In [ ]:
feature = df_dnsampled[['WEATHER', 'ROADCOND', 'LIGHTCOND']]
y = df_dnsampled['SEVERITYCODE'].values


cleanup_nums = {'WEATHER':    {"Clear": 0, 'Raining': 1, 'Overcast': 2, 'Unknown': 3,
                              'Snowing': 4, 'Other': 5, 'Fog/Smog/Smoke': 6, 'Sleet/Hail/Freezing Rain': 7,
                              'Blowing Sand/Dirt': 8, 'Severe Crosswind': 9, 'Partly Cloudy': 10},
                'ROADCOND':   {"Dry": 0, "Wet": 1, "Unknown": 2, "Ice": 3,
                              "Snow/Slush": 4, "Other": 5, "Standing Water": 6,
                              'Sand/Mud/Dirt': 7, 'Oil': 8},
               'LIGHTCOND': {'Daylight': 0, 'Dark - Street Lights On': 1, 'Unknown': 2, 'Dusk': 3, 
                            'Dawn': 4, 'Dark - No Street Lights': 5, 'Dark - Street Lights Off': 6,
                            'Other': 7, 'Dark - Unknown Lighting':8}}

feature.replace(cleanup_nums, inplace=True)

X = feature

In [ ]:
X = np.asarray(X[['WEATHER', 'ROADCOND', 'LIGHTCOND']])
X[0:5]
